In [1]:
from Crypto.Cipher import AES
from Crypto.Random import random
from hashlib import sha256

ca_key = (123911229913422142875264410911959753429255609781060012540592865384481400430784339515602026823919372067245349130530370079412035001372557207523878469898937327849288805682100737656179353201361997603827751534685210816357362467822736133251729781421390068496293277151621728336136046748401714936280146300069794792289, 65537)

P = 90217665064898209874945440415607645619739552209209829698859302616784804764535733761793578496057231124307702060703777047113648993244620436073319613409562635176198907683514151487667596066485528729421061091099275256268502121631878101098253283229895343146346757918637948217208979264467987857303672675468072062419

G = 13

certs = ["416c696365_7d2e831fc7c37b4932604c66de52a3c7f631eee0badffb42f40899fb88ff19e41518fc8326f620aaeb54cecd8bd22a5fa8576848c131370c0fcb8fb0e093fbd0d76b73c47c04ffe2b913472595c9987d53b8cdffa4ff16f6a6176f215e6ded63e65089d763691a4cef91284e6934b95580e64854304b99e87215b582e201222d", "426f62_2453e551e17519b6dcfdae46f75b9d92dd943fcb5d2c71911adb83f43753dfba86be013dfb8c2413cbc70eac852e312040d3fbf471b57da2ae7a7aa95d39f3b972e34214f5cc69093d9929b2c5fc24684b0d5443dcb8338d1f52bb34f9913bd1e98c29302bbbd3cb532001adde7f06d93badcdee7b12b1b550a816537e950bad"]

def pad(msg):
    n = 16 - len(msg) % 16
    return msg + bytes([n]) * n

def unpad(msg):
    assert len(msg) > 0 and len(msg) % 16 == 0
    n = msg[-1]
    assert 1 <= n <= 16
    assert msg[-n:] == bytes([n]) * n
    return msg[:-n]

In [2]:
fromAliceAndBob1 = [
    "25edd0be57f797d1abda2708737876c8824578b274e5052d243e747f26f92147aa49b21d18cf1956f35093634fdc75b7483540fac376875ba34cf3a607463f2e7d3635330552e027f1474ba391c131aff90ce024f5026a538b90011444889449bf4e6f264fdc670172c687b26fdc61f0d471eb272fa5b0a7126592540a680222",
    "602c96cfc57071a851ca3a530982179d8c2b685fb18558f2037a670766404a8568037b986ca3d207b7c5f314b0c7800c8b4b6cf0bbdf831451ed68413ac3e579a4fa338d55680227ca1c9e6c9ad07286a4f244e5f7c73ded3ca4cab721c3133af47d23e863ca79328118181a2d5bd7ad0b148cb84b336e10c00e0675d03c0ad9"
]

# myKey = random.getrandbits(1024)
myKey = 0
myKey -= myKey & 1

keys = [pow(int(from1, 16), myKey, P) for from1 in fromAliceAndBob1]
aesKeys = [int.to_bytes(key % (2**128), 16, "big") for key in keys]

toAliceAndBob1 = [hex(pow(G, myKey, P))[2: ] for _ in fromAliceAndBob1]

print("toAliceAndBob1 == {}".format(toAliceAndBob1))

toAliceAndBob1 == ['1', '1']


In [3]:
fromAliceAndBob2 = [
    "0f9a2f25b11b3568fc8813cc47be92509129b7bc1ea0cb763ff8d561bb7b67734829b745d3ba2f7bf32a56c45093285e2c2977a6459a140d1f4464e46043a7ec841e21593b761cfbbb7f334ad75d640e99b3454dc0ab1cb83c3fc63441b5cc42c960f82753560b0f5017e2a43dd11c05adbe6e6c22ece721bc56cb7a3ca27f80f805115d3f9972a17e3249be3f782d231b44e5755f6b12def52d8b2a09b75666dfee3c1e2ce242b6e08acda6d8ea778e86352ea370baf0a744446e45c444781f58edd76c3c0d6d7991a00c78924b5a3faa259754cf399c990126261aee6f44aefdcdb84fc7e932786998953c2c76997f368bc7a646aedad0a76dfd83d71d38423e4618491992d3e7f4e6c5ac91c097b10ebcfc0a0bcae53b512855dd6edd5e0b",
    "886d8524f0f1d26d15fab437b0f633c018b2883930f506360b0792e23dbe89cf59a74cabc515b898b0757bd0157149a78f59a5d8370b4aae6d0607ec5dc336cc13d276d8a856da075028a919a5bbcace1bd29f8311b3d3e4e42efed9ea323594d84078861dd53c457c52603bbf2840ecedd5fbcb13e85a33281b5e2cfb594004313a9a22c2b603742dfb8d62d01ba8d28eecfb62791decd6454e058d2d1238d16fa3afadd28738a38cbb5b696577f8aab070a1522e68c0ee77cd12a32b950c618ba3d623851bc5a9d88ede4c794fdf6ef05764d53673dc3dec5b814b899b835c7d74d929796017e56764fa3976421b382bc5d742667c0507f4995c19d02632adccc5b5313d84fe45668a83a3cb585f90ba84d80222b4d85f50d031c94cd36d5b"
]

aesIvsAndCiphers = [bytes.fromhex(from2) for from2 in fromAliceAndBob2]
aesIvs = [aesIvAndCipher[: 16] for aesIvAndCipher in aesIvsAndCiphers]
ciphers = [aesIvAndCipher[16: ] for aesIvAndCipher in aesIvsAndCiphers]
aeses = [AES.new(aesKey, AES.MODE_CBC, aesIv) for aesKey, aesIv in zip(aesKeys, aesIvs)]

toAlice2 = [(aesIv + aes.encrypt(pad(cert.encode()))).hex() for aesIv, aes, cert in zip(aesIvs, aeses, reversed(certs))][0]

print("toAlice2 == {}".format(toAlice2))

toAlice2 == 0f9a2f25b11b3568fc8813cc47be9250d575dbc8272ac0370a344dd6df6d7325cf4452cb74f58cec001d2a8686a552f7861cab9052c9f7b39016d539e2924c864993c13f02502b513ebd8c4fbbc478b2005e58d0a22989c5a9319d585fc19e788bb1f13f91fe674b3ba0200b6055aa5feb13d0896517f8fc82a4414031d8a471ffb7e7af55f303230011f60819c276fa8abaa668b89ab7a2162fd758e29955ab9a189dbdee746426852ebe90c43775f8b9ca7e68773b9762b3d0a375ad78fa9f9e30ef95eeaa983a1e2f7d0263a75e79552cc83ed55c7143dec8d43a2fa936355a48fb87a4a96b699a594500085c15d92fda9c1e7601d5e2929da8f4c33f8f8de8c0516ad385e647e33d9221f8c1d9c80cf7e69bad870bca88e4ee9af0918d92


In [4]:
fromAlice3 = "0b253f6be7dd61c572f431f7b5537ec254365dbc71b37611298f95f430fb36a0e10b375e83c72eea63cd80ff6e02d6a6b0340cf9d492b9dc288c36389fd8f4cc"

aesIvAndCipher = bytes.fromhex(fromAlice3)
aesIv, cipher = aesIvAndCipher[: 16], aesIvAndCipher[16: ]
aes = AES.new(aesKeys[0], AES.MODE_CBC, aesIv)
message = unpad(aes.decrypt(cipher)).decode()
flag = message

print("flag == {}".format(flag))
print("key == {}".format(keys[0]))
print("(key < 2**512) == {}".format(keys[0] < 2**512))
print("flag is: {}".format("flag1" if keys[0] < 2**512 else "flag2"))

flag == flag{th3_Tr1V14l_1s_N0t_trIviAL_920f454e}
key == 1
(key < 2**512) == True
flag is: flag1


In [5]:
flag = 'flag{th3_Tr1V14l_1s_N0t_trIviAL_920f454e}'
key = 1
(key < 2**512) == True
'flag is: flag1'

'flag is: flag1'